In [1]:
import requests
import json
from pandas.io.json import json_normalize
import pandas as pd
import time
import logging
import numpy as np
import collections, re
from collections import OrderedDict
import datetime

In [3]:
#List of genres
genredata = requests.get("http://api.themoviedb.org/3/genre/movie/list?language=en-US&api_key=d34bc9fba7aa52b8156fd52d8f59bd08")
genredetails = genredata.json()
genre_df = json_normalize(genredetails['genres'])
genre_df

,id,name
0,28,Action
1,12,Adventure
2,16,Animation
3,35,Comedy
4,80,Crime
5,99,Documentary
6,18,Drama
7,10751,Family
8,14,Fantasy
9,36,History


In [4]:
#Fetch the dataset by genre
def getmoviesbygenre(movie_df, key, genre_id, year, count = 1000):
    data_size = requests.get("http://api.themoviedb.org/3/discover/movie?with_genres="
                             +genre_id+"&primary_release_year="+year+"&api_key="+key)
    temp_df = json_normalize(data_size.json())
    sleepcount = 0
    for i in range (1, temp_df.iloc[0][2]+1):
        if(len(movie_df) >= count):
            break
        if(sleepcount == 30):
            sleepcount = 0
            time.sleep(15)
        json_request = requests.get("http://api.themoviedb.org/3/discover/movie?with_genres="
                             +genre_id+"&primary_release_year="+year+"&page="+str(i)+"&api_key="+key)
        movie_df = movie_df.append(json_normalize(json_request.json()['results']))
        sleepcount = sleepcount + 1        
    return movie_df                                

In [46]:
#wrapper function for fetching the genres
def callgmbg():
    key ='d34bc9fba7aa52b8156fd52d8f59bd08'
    movie_df = pd.DataFrame()
    genre_id = [27, 35, 18, 28, 9648, 878, 10749, 36]
    for i in genre_id:
        movie_temp_df = pd.DataFrame()
        year = 2018
        while(len(movie_temp_df) < 3000):
            time.sleep(10)
            year = year - 1
            movie_temp_df = getmoviesbygenre(movie_temp_df, key, str(i), str(year), 3000)
            movie_df = movie_df.append(movie_temp_df)
    return movie_df

In [6]:
#Function to map genre names
def find_genre(genre_ids):
    in_genre = []
    for i in range(0, len(genre_ids)):
        if(genre_ids[i] in list(genre_df.id)):
            in_genre.append(genre_df.loc[genre_df['id'] == genre_ids[i], 'name'].item())
        else:
            continue
    return in_genre

In [47]:
#create genre dataset -- only this needs to be executed to get the final dataset
movie_df = callgmbg()
movie_df= movie_df.reset_index(drop=True)
movie_df["tmdb_genre"] = movie_df["genre_ids"].apply(find_genre)

In [58]:
def getkeywords(df):
    cnt = 0
    df['keywords'] = 'na'
    for i in range(0, len(df)):
        try:
            if(cnt == 39):
                time.sleep(15)
                cnt = 0
            cnt = cnt + 1
            data1 = requests.get("https://api.themoviedb.org/3/movie/" + str(df.loc[i].id) +"/keywords?api_key=d34bc9fba7aa52b8156fd52d8f59bd08")
            if('keywords' in data1.json() and len(data1.json()['keywords']) > 0):
                df.set_value(i, 'keywords', list(json_normalize(data1.json()['keywords'])['name']))
            else:
                continue
        except Exception:
            continue
    return df 

In [57]:
def getmoviedata(df):
    counter = 0
    df['imdb_id'] = 'na'
    df['revenue'] = 'na'
    df['runtime'] = 'na'
    for i in range(0, len(df)):
        try:
            if(counter == 39):
                counter = 0
                time.sleep(15)
            #print df.loc[i].id
            counter = counter + 1
            data1 = requests.get("https://api.themoviedb.org/3/movie/" + str(df.loc[i].id) +"?api_key=d34bc9fba7aa52b8156fd52d8f59bd08&language=en-US")
            if('imdb_id' in data1.json() and json_normalize(data1.json())['imdb_id'][0] is not None):
                df.set_value(i, 'imdb_id', json_normalize(data1.json())['imdb_id'][0][2:])
            else:
                continue
            if('revenue' in data1.json() and json_normalize(data1.json())['revenue'][0] is not None):
                df.set_value(i, 'revenue', json_normalize(data1.json())['revenue'][0])
            else:
                continue
            if('runtime' in data1.json() and json_normalize(data1.json())['runtime'][0] is not None):
                df.set_value(i, 'runtime', json_normalize(data1.json())['runtime'][0])
            else:
                continue
        except Exception:
            continue
    return df

In [53]:
print len(movie_df)
print len(np.unique(movie_df.id))
movie_df = movie_df.drop_duplicates('id')
print len(movie_df)
print len(np.unique(movie_df.id))

178164
19907
19907
19907


In [ ]:
start_time = time.time()
movie_temp_df = getmoviedata(movie_df)
print("--- %s seconds ---" % (time.time() - start_time))
movie_temp_df.to_csv("movie_df.csv", sep=',', encoding='utf-8',index =False,escapechar ='\\')
start_time = time.time()
movie_temp_df = getkeywords(movie_temp_df)
print("--- %s seconds ---" % (time.time() - start_time))
movie_temp_df.to_csv("movie_df_final.csv", sep=',', encoding='utf-8',index =False,escapechar ='\\')

In [ ]:
###### movie_temp_df.to_csv("moviedf_final.csv", sep=',', encoding='utf-8',index =False,escapechar ='\\')

In [42]:
temp = pd.read_csv("moviedf_final.csv", sep=',', encoding='utf-8')

In [61]:
np.shape(movie_temp_df)

(19907, 19)

In [99]:
import csv
with open(r"movies_allfeatures.csv", 'rb') as f:
    reader = csv.reader(f)
    linenumber = 1
    try:
        for row in reader:
            linenumber += 1
    except Exception as e:
        print (("Error line %d: %s %s" % (linenumber, str(type(e)), e.message)))

In [103]:
movie_stitched_df = pd.read_csv("movies_allfeaturesnew.csv", sep=',', encoding='utf-8',escapechar ='\\' )
print np.shape(movie_stitched_df)
movie_stitched_df = movie_stitched_df.drop_duplicates('id')
print np.shape(movie_stitched_df)
movie_stitched_df = movie_stitched_df.reset_index(drop=True)

(5695, 76)
(5695, 76)


In [95]:
biglist = []
stopwords = ['director', 'movie', 'novel', 'new','in', 'on', 'of', 'the', 'to', 'it', 'for', 'na', 'based', 'the', 'book']
def createbiglist(texts):
    for txt in texts.split():
        if txt not in stopwords:
            biglist.append(txt)
        else:
            continue

In [96]:
movie_stitched_df['keywords'].apply(createbiglist)
bagsofwords = [collections.Counter(re.findall(r'\w+', txt))
                    for txt in biglist]
bagsofwords
sumbags = sum(bagsofwords, collections.Counter())
sorted_set = OrderedDict(sorted(sumbags.items(), 
                                  key=lambda kv: kv[1], reverse=True))
for key in sorted_set.keys()[0:40]:
    movie_stitched_df[key] = 0

for i in range(0, len(movie_stitched_df)):
    for key in sorted_set.keys()[0:40]:
        if key in movie_stitched_df.keywords[i]:
            movie_stitched_df[key][i] = 1
        else:
            continue
movie_stitched_df.head()

/home/cs109b/.local/lib/python2.7/site-packages/ipykernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,...,thriller,gay,book,post,supernatural,future,suspense,found,superhero,footage
0,False,/4G6FNNLSIVrwSRZyFs91hQ3lZtD.jpg,"[27, 53]",381288,en,Split,Though Kevin has evidenced 23 personalities to...,42.679724,/rXMWOZiCt6eMX22jWuTOSdQ98bY.jpg,1/19/2017,...,0,0,0,0,0,0,0,0,0,0
1,False,/hES8wGmkxHa54z7hqUMpw5TIs09.jpg,"[27, 878, 53]",395992,en,Life,The six-member crew of the International Space...,20.107715,/h2mhfbEBGABSHo2vXG1ECMKAJa7.jpg,3/23/2017,...,0,0,0,0,0,0,0,0,0,0
2,False,/biN2sqExViEh8IYSJrXlNKjpjxx.jpg,[27],14564,en,Rings,"Julia becomes worried about her boyfriend, Hol...",7.472619,/bbxtz5V0vvnTDA2qWbiiRC77Ok9.jpg,2/1/2017,...,0,0,0,0,0,0,1,0,0,0
3,False,/kMU8trT43p5LFoJ4plIySMOsZ1T.jpg,"[12, 28, 27, 878, 53]",126889,en,Alien: Covenant,Bound for a remote planet on the far side of t...,6.149548,/kwYzMCNUWUADZo9ox2jbUbcNT7O.jpg,5/10/2017,...,0,0,0,0,0,0,0,0,0,0
4,False,/pJgoSw2tdV0vrjd2v9IreWFWXHM.jpg,"[27, 53]",292280,en,The Bye Bye Man,When three college students move into an old h...,4.828029,/m5HSTW3YTOu4b0TZUA6cbroOl2J.jpg,1/6/2017,...,0,0,0,0,1,0,0,0,0,0


In [104]:
movie_stitched_df['genre_category'] = movie_stitched_df.tmdb_genre.apply(generateresponse)
movie_stitched_df.head(n = 50)

,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,...,post,supernatural,future,suspense,found,superhero,footage,brightness,brightness_level,genre_category
0,False,/4G6FNNLSIVrwSRZyFs91hQ3lZtD.jpg,"[27, 53]",381288,en,Split,Though Kevin has evidenced 23 personalities to...,42.679724,/rXMWOZiCt6eMX22jWuTOSdQ98bY.jpg,1/19/2017,...,0,0,0,0,0,0,0,22.1562,Low,AcCrTh
1,False,/hES8wGmkxHa54z7hqUMpw5TIs09.jpg,"[27, 878, 53]",395992,en,Life,The six-member crew of the International Space...,20.107715,/h2mhfbEBGABSHo2vXG1ECMKAJa7.jpg,3/23/2017,...,0,0,0,0,0,0,0,16.7404,Low,AcCrTh
2,False,/biN2sqExViEh8IYSJrXlNKjpjxx.jpg,[27],14564,en,Rings,"Julia becomes worried about her boyfriend, Hol...",7.472619,/bbxtz5V0vvnTDA2qWbiiRC77Ok9.jpg,2/1/2017,...,0,0,0,1,0,0,0,44.0608,Low,Ho
3,False,/kMU8trT43p5LFoJ4plIySMOsZ1T.jpg,"[12, 28, 27, 878, 53]",126889,en,Alien: Covenant,Bound for a remote planet on the far side of t...,6.149548,/kwYzMCNUWUADZo9ox2jbUbcNT7O.jpg,5/10/2017,...,0,0,0,0,0,0,0,0.7152,Low,AcCrTh
4,False,/pJgoSw2tdV0vrjd2v9IreWFWXHM.jpg,"[27, 53]",292280,en,The Bye Bye Man,When three college students move into an old h...,4.828029,/m5HSTW3YTOu4b0TZUA6cbroOl2J.jpg,1/6/2017,...,0,1,0,0,0,0,0,11.3346,Low,AcCrTh
5,False,/6oXUTHVAjPrNKvVFZhxNlWv7jua.jpg,"[18, 27, 9648, 53]",340837,en,A Cure for Wellness,An ambitious young executive is sent to retrie...,4.574548,/byeTgTgG7M1RN2c7njWWIkSkNig.jpg,2/15/2017,...,0,0,0,0,0,0,0,158.8408,Medium,AcThMi
6,False,/9bCzBwdmtV53zucixT8uuz0C5ia.jpg,[27],378018,en,The Void,"In the middle of a routine patrol, officer Dan...",4.273311,/nH1YZAL8IeIQEOm2ewi5ncpuPQG.jpg,4/7/2017,...,0,0,0,0,0,0,0,37.2170,Low,Ho
7,False,/zFEPvuTpsOjsE4571npAMoAZknB.jpg,[27],346364,en,It,"In a small town in Maine, seven children known...",3.392068,/9E2y5Q7WlCVNEhP5GiVTjhEhx1o.jpg,9/7/2017,...,0,0,0,0,0,0,0,18.6378,Low,Ho
8,False,/4XfP8wNIA1GBShNIzlSb6SSDCex.jpg,"[28, 27, 878, 53]",341006,en,The Belko Experiment,"In a twisted social experiment, a group of 80 ...",3.024100,/knAqP7qDos8sWQpmPiGYFYwbdcK.jpg,3/17/2017,...,0,0,0,0,0,0,0,190.8544,High,AcCrTh
9,False,/osuawAz6tjeaZq7R1mlTwIwTj4H.jpg,"[28, 12, 14, 27, 53]",282035,en,The Mummy,Though safely entombed in a crypt deep beneath...,2.828962,/zxkY8byBnCsXodEYpK8tmwEGXBI.jpg,6/8/2017,...,0,0,0,0,0,0,0,31.6430,Low,AcCrTh


In [107]:
movie_stitched_df.to_csv("movies_allfeaturesfinal.csv", sep=',', encoding='utf-8',index =False,escapechar ='\\')